In [170]:
import pandas as pd
df = pd.read_csv('Data/Wcr_GPPdaily.csv') # Read dataset
df.head()

,TIMESTAMP,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,19990101,-20.063,72.603,0.413,-0.517364
1,19990102,-12.814,12.358,0.147,-0.094241
2,19990103,-12.625,33.132,0.128,-0.166819
3,19990104,-18.652,93.481,0.263,-0.582301
4,19990105,-20.269,45.502,0.261,-0.568240


In [172]:
df = df.rename(columns = {'TA_F': 'TA','SW_IN_F': 'SW', 'VPD_F': 'VPD', 'GPP_NT_VUT_REF': 'GPP'}) # Rename columns
df = df.drop('TIMESTAMP', axis=1) # Drop TIMESTAMP column
df.head()

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [178]:
print(df.isnull().sum()) # No missing values
print(df.isnull().any().any())

TA     0
SW     0
VPD    0
GPP    0
dtype: int64
False


In [180]:
# Compute interaction terms and store them in dataframe column
df['SW×VPD'] = df.SW * df.VPD
df['SW×TA'] = df.SW * df.TA
df['VPD×TA'] = df.VPD * df.TA
df.head()

,TA,SW,VPD,GPP,SW×VPD,SW×TA,VPD×TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209


In [184]:
import numpy as np
from scipy.optimize import minimize

# Define the linear model function
def linear_model(params, SW, VPD, TA):
    beta0, beta1, beta2, beta3, beta4, beta5, beta6 = params
    return (beta0 + beta1*SW + beta2*VPD + beta3*TA + 
            beta4*(SW*VPD) + beta5*(SW*TA) + beta6*(VPD*TA))

# Define the Mean Squared Error function
def mse(params, SW, VPD, TA, GPP):
    predictions = linear_model(params, SW, VPD, TA)
    return np.mean((GPP - predictions)**2)

# Prepare the data
SW = np.array(df['SW'])
VPD = np.array(df['VPD'])
TA = np.array(df['TA'])
GPP = np.array(df['GPP'])

# Initial parameter guess
initial_params = np.zeros(7)

# Optimize the parameters
result = minimize(mse, initial_params, args=(SW, VPD, TA, GPP))

# Print results
print("Optimized coefficients:")
print(f"β0 (Intercept): {result.x[0]:.4f}")
print(f"β1 (SW): {result.x[1]:.4f}")
print(f"β2 (VPD): {result.x[2]:.4f}")
print(f"β3 (TA): {result.x[3]:.4f}")
print(f"β4 (SW×VPD): {result.x[4]:.4f}")
print(f"β5 (SW×TA): {result.x[5]:.4f}")
print(f"β6 (VPD×TA): {result.x[6]:.4f}")

Optimized coefficients:
β0 (Intercept): -944.0438
β1 (SW): -3.2481
β2 (VPD): -648.7368
β3 (TA): 82.1823
β4 (SW×VPD): 1.7928
β5 (SW×TA): -0.4562
β6 (VPD×TA): 13.5997


In [196]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Prepare features (X) and target variable (y)
X = df[['TA', 'SW', 'VPD', 'SW×VPD', 'SW×TA', 'VPD×TA']]  # Set features (independent variables)
y = df['GPP'] # Set target variable to GPP

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Print the model coefficients
print("Optimized coefficients:")
print(f"β0 (Intercept): {model.intercept_:.4f}")
print(f"β1 (SW): {model.coef_[1]:.4f}")
print(f"β2 (VPD): {model.coef_[2]:.4f}")
print(f"β3 (TA): {model.coef_[0]:.4f}")
print(f"β4 (SW×VPD): {model.coef_[3]:.4f}")
print(f"β5 (SW×TA): {model.coef_[4]:.4f}")
print(f"β6 (VPD×TA): {model.coef_[5]:.4f}")

Optimized coefficients:
β0 (Intercept): -963.0331
β1 (SW): -3.0946
β2 (VPD): -715.4821
β3 (TA): 84.8424
β4 (SW×VPD): 1.9977
β5 (SW×TA): -0.4938
β6 (VPD×TA): 15.5562


In [198]:
def r_squared(params, SW, VPD, TA, GPP):
    # Get predictions
    predictions = linear_model(params, SW, VPD, TA)
    
    # Calculate R-squared
    ss_total = np.sum((GPP - np.mean(GPP))**2)  # Total sum of squares
    ss_residual = np.sum((GPP - predictions)**2)  # Residual sum of squares
    r2 = 1 - (ss_residual / ss_total)
    
    return r2

# Calculate and print R-squared
r2_value = r_squared(result.x, SW, VPD, TA, GPP)
print(f"\nR-squared of linear regression model using SciPy Optimization: {r2_value:.4f}")

print(f"\nR-squared of linear regression model using Sklearn's Linear Regression Function : {model.score(X_test, y_test):.4f}")

# Calculate and print the difference in R-squared
difference = r2_value - model.score(X_test, y_test)
print(f"\nDifference: {difference:.4f}") 

# The R-squared value using sklearns's linear regression model is smaller than the R-squared value using SciPy Optimization by 0.0073.
# This may indicate that the linear regression model using SciPy Optimization is a better-fit model of the data.
# However, it must be noted that both R-squared values are relatively small, so both models may not be a good fit of the data.


R-squared of linear regression model using SciPy Optimization: 0.0136

R-squared of linear regression model using Sklearn's Linear Regression Function : 0.0062

Difference: 0.0073


In [200]:
# Comparison of coeffictients. 
print("Difference in coefficients:") # The below code computes the difference between the SciPy model and the Sklearn model
B0 = result.x[0] - model.intercept_
B1 = result.x[1] - model.coef_[1]
B2 = result.x[2] - model.coef_[2]
B3 = result.x[3] - model.coef_[0]
B4 = result.x[4] - model.coef_[3]
B5 = result.x[5] - model.coef_[4]
B6 = result.x[6] - model.coef_[5]
print(f"Difference in β0 (Intercept): {B0: .4f}") #Print differences
print(f"Difference in β1 (SW): {B1: .4f}")
print(f"Difference in β2 (VPD): {B2: .4f}")
print(f"Difference in β3 (TA): {B3: .4f}")
print(f"Difference in β4 (SWxVPD): {B4: .4f}")
print(f"Difference in β5 (SWxTA): {B5: .4f}")
print(f"Difference in β6 (VPDxTA): {B6: .4f}")

Difference in coefficients:
Difference in β0 (Intercept):  18.9892
Difference in β1 (SW): -0.1535
Difference in β2 (VPD):  66.7453
Difference in β3 (TA): -2.6601
Difference in β4 (SWxVPD): -0.2049
Difference in β5 (SWxTA):  0.0377
Difference in β6 (VPDxTA): -1.9565
